In [11]:
%pip install transformers
%pip install torch
%pip install osmnx

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 99.6/99.6 kB 1.9 MB/s eta 0:00:00
     -------------------------------------- 323.6/323.6 kB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 1.4/1.4 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 16.2/16.2 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 6.3/6.3 MB 3.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip available: 22.3 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
import numpy as np
import osmnx as ox
import matplotlib.pyplot as plt
import math

In [13]:
ev_df = pd.read_csv("./eVED/eVED_181107_week.csv")
ev_df.head()

FileNotFoundError: [Errno 2] No such file or directory: './eVED/eVED_181107_week.csv'

In [ ]:
grouped = ev_df.groupby(['VehId', 'Trip'])

# Extract complete routes as sequences
route_sequences = []
routes_total = 0;

for (vehid, trip), group in grouped:
    # Sort by timestamp to ensure correct sequence
    group = group.sort_values('Timestamp(ms)')

    # Extract coordinates as sequence
    route = list(zip(group['Latitude[deg]'], group['Longitude[deg]']))

    # Store as (source, destination, full_route)
    source = route[0]
    destination = route[-1]
    route_sequences.append({
        'source': source,
        'destination': destination,
        'full_route': route,
        'vehicle': vehid,
        'trip': trip,
    })

    routes_total += 1


print(routes_total)

170


In [ ]:
def create_features(route_data):
    features = []
    for data in route_data:
        src_lat, src_lng = data['source']
        dst_lat, dst_lng = data['destination']

        feature = f"[SRC] {src_lat:.6f} {src_lng:.6f} [DST] {dst_lat:.6f} {dst_lng:.6f}"

        features.append(feature)

    return features


In [14]:
from torch.utils.data import Dataset, DataLoader

class RouteDataset(Dataset):
  def __init__(self, route_data, tokenizer, max_points=100):
    self.route_data = route_data
    self.tokenizer = tokenizer
    self.max_points = max_points
    self.features = create_features(route_data)
    self.route_targets = self.tokenize_routes(route_data)

  def __len__(self):
    return len(self.route_data)

  def __getitem__(self, idx):
    item_ft = self.features[idx]
    item_tokenized = self.tokenizer(
        item_ft, padding="max_length", truncation=True, max_length=128,
        return_tensors="pt"
    )

    target = self.route_data[idx]
    item = {
        'input_ids': item_tokenized['input_ids'].squeeze(),
        'attention_mask': item_tokenized['attention_mask'].squeeze(),
        'target_routes': torch.tensor(target, dtype=torch.float32)
    }

    return item

  def tokenize_routes(self, route_data):
    tokenized_routes = []
    print(route_data)
    for data in route_data:
        route = data['full_route']
        if len(route) > self.max_points:
            indices = np.linspace(0, len(route)-1, self.max_points).astype(int)
            route = [route[i] for i in indices]
        else:
            route = route + [(0, 0)] * (self.max_points - len(route))

        flat_route = []
        for lat, lng in route:
            flat_route.append(lat)
            flat_route.append(lng)

        tokenized_routes.append(flat_route)
    return tokenized_routes


In [15]:
def get_dataloader(route_data, tokenizer, batch_size=8, shuffle=True, max_points=100):
  dataset = RouteDataset(route_data, tokenizer, max_points=max_points)
  loader = DataLoader(
      dataset, batch_size=batch_size,
      shuffle=shuffle,
      num_workers=2
  )

  return loader

In [16]:
from transformers import BertModel, BertConfig, BertTokenizer
import torch.nn as nn
import torch

class RoutePredictor(nn.Module):
    def __init__(self, hidden_size=768, max_points=100):
      super().__init__()
      self.config = BertConfig(
          vocab_size=30522,
          hidden_size=hidden_size,
          num_hidden_layers=6,
          num_attention_heads=8,
          intermediate_size=hidden_size*4
      )
      self.bert = BertModel(self.config)
      self.route_head = nn.Linear(hidden_size, max_points * 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_output = outputs.last_hidden_state[:, 0]  # Use [CLS] token representation
        route_prediction = self.route_head(cls_output)
        return route_prediction.view(route_prediction.size(0), -1, 2)  # Reshape to (batch, points, 2)


RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
module 'torch' has no attribute 'compiler'

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(route_sequences, test_size=0.2, random_state=42)

def train_model(model, train_data, val_data, epochs=10):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    train_loader = get_dataloader(train_data, tokenizer)
    val_loader = get_dataloader(val_data, tokenizer)

    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
    loss_fn = nn.MSELoss()

    for epoch in range(epochs):
      model.train()
      train_loss = 0

      for batch in train_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        target_routes = batch['target_routes']

        outputs = model(input_ids, attention_mask)
        loss = loss_fn(outputs, target_routes)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

      model.eval()
      val_loss = 0

      with torch.no_grad():
          for batch in val_loader:
              input_ids = batch['input_ids']
              attention_mask = batch['attention_mask']
              target_routes = batch['target_routes']

              outputs = model(input_ids, attention_mask)
              loss = loss_fn(outputs, target_routes)
              val_loss += loss.item()

      print(f"Epoch {epoch+1}/{epochs}")
      print(f"Train Loss: {train_loss/len(train_loader):.4f}")
      print(f"Val Loss: {val_loss/len(val_loader):.4f}")

In [ ]:
model = RoutePredictor()
train_model(model, route_sequences[:150], route_sequences[-20:])

[{'source': (42.2954772222, -83.7050052778), 'destination': (42.2807525, -83.7510227778), 'full_route': [(42.2954772222, -83.7050052778), (42.2954772222, -83.7050052778), (42.2954772222, -83.7050052778), (42.2954772222, -83.7050052778), (42.2954772222, -83.7050052778), (42.2951991667, -83.7049566667), (42.2951991667, -83.7049566667), (42.2951991667, -83.7049566667), (42.2951991667, -83.7049566667), (42.2951991667, -83.7049566667), (42.2951991667, -83.7049566667), (42.2951991667, -83.7049566667), (42.29487, -83.7049425), (42.29487, -83.7049425), (42.29487, -83.7049425), (42.29487, -83.7049425), (42.29487, -83.7049425), (42.29487, -83.7049425), (42.29487, -83.7049425), (42.2944786111, -83.7049669444), (42.2944786111, -83.7049669444), (42.2944786111, -83.7049669444), (42.2944786111, -83.7049669444), (42.2944786111, -83.7049669444), (42.2944786111, -83.7049669444), (42.2944786111, -83.7049669444), (42.2944786111, -83.7049669444), (42.2944786111, -83.7049669444), (42.2939241667, -83.7051069

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "<ipython-input-18-0f3a13d4f878>", line 25, in __getitem__
    'target_routes': torch.tensor(target, dtype=torch.float32)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: must be real number, not dict


In [ ]:
def predict_route(model, source, destination):
    # Prepare input like during training
    feature = f"[SRC] {source[0]:.6f} {source[1]:.6f} [DST] {destination[0]:.6f} {destination[1]:.6f}"

    # Tokenize, convert to tensors
    inputs = tokenizer(feature, return_tensors="pt")

    # Get prediction
    with torch.no_grad():
        route_coords = model(**inputs)

    # Convert from tensor to list of coordinates
    predicted_route = [(lat.item(), lng.item()) for lat, lng in route_coords[0]]

    return predicted_route

In [ ]:
# Visualize a predicted route
def visualize_route(source, destination, predicted_route):
    # Get map bounds
    lats = [coord[0] for coord in predicted_route] + [source[0], destination[0]]
    lngs = [coord[1] for coord in predicted_route] + [source[1], destination[1]]

    north, south = max(lats) + 0.01, min(lats) - 0.01
    east, west = max(lngs) + 0.01, min(lngs) - 0.01

    # Create smaller, simplified graph
    G = ox.graph_from_bbox(north, south, east, west, network_type='drive_service', simplify=True)

    # Plot
    fig, ax = plt.subplots(figsize=(10, 10))
    ox.plot_graph(G, ax=ax, show=False, close=False, node_size=0, edge_linewidth=0.5)

    # Plot predicted route
    pred_lats = [coord[0] for coord in predicted_route]
    pred_lngs = [coord[1] for coord in predicted_route]
    ax.plot(pred_lngs, pred_lats, 'r-', linewidth=2, label='Predicted')

    # Plot source/destination
    ax.plot(source[1], source[0], 'go', markersize=10, label='Source')
    ax.plot(destination[1], destination[0], 'ro', markersize=10, label='Destination')

    ax.legend()
    plt.tight_layout()
    plt.show()